## Method 1

In [1]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

C:\Users\vijja\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [3]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi")

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [5]:
raw_datasets['train'][10]

{'translation': {'en': 'The color and opacity of the highlight fill.',
  'hi': 'हाइलाइट किया गया भराई का रंग और पारदर्शिता। '}}

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

C:\Users\vijja\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [16]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [12110, 2, 90, 23, 19, 8800, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[12110, 2, 90, 23, 19, 8800, 61, 0], [239, 23, 414, 8800, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [18]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["एक्सेर्साइसर पहुंचनीयता अन्वेषक"]))

{'input_ids': [[26618, 16155, 346, 33383, 0]], 'attention_mask': [[1, 1, 1, 1, 1]]}


In [19]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "hi"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [20]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0]]}

In [21]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [22]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [23]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [24]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [25]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [26]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)


In [27]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [28]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [29]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [30]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Metric to monitor (e.g., validation loss)
    patience=5,          # Number of epochs with no improvement to wait
    restore_best_weights=True  # Restore model weights to the best achieved during training
)

In [31]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=20,callbacks=[early_stopping_callback])

Epoch 1/20
156/156 [==============================] - 917s 6s/step - loss: 3.7659 - val_loss: 3.9449
Epoch 2/20
156/156 [==============================] - 925s 6s/step - loss: 3.3246 - val_loss: 3.8610
Epoch 3/20
156/156 [==============================] - 1343s 9s/step - loss: 3.0191 - val_loss: 3.8367
Epoch 4/20
156/156 [==============================] - 1387s 9s/step - loss: 2.7759 - val_loss: 3.8238
Epoch 5/20
156/156 [==============================] - 1264s 8s/step - loss: 2.5697 - val_loss: 3.8179
Epoch 6/20
156/156 [==============================] - 1360s 9s/step - loss: 2.3817 - val_loss: 3.8223
Epoch 7/20
156/156 [==============================] - 9552s 62s/step - loss: 2.2055 - val_loss: 3.8346
Epoch 8/20
156/156 [==============================] - 2433s 16s/step - loss: 2.0559 - val_loss: 3.8350
Epoch 9/20
156/156 [==============================] - 701s 5s/step - loss: 1.9161 - val_loss: 3.8595
Epoch 10/20
156/156 [==============================] - 1341s 9s/step - loss: 1.7726

In [26]:
model.save_pretrained("tf_model/")

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

C:\Users\vijja\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [31]:
input_text  = "My name vinay, how are you."

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[61949   500   179  6206  7668   260     2   118   280    28    40     0
  61949 61949]], shape=(1, 14), dtype=int32)


In [32]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

मेरा नाम विधा, आप कैसे हैं।


## Method2

Using Large Language Model

In [5]:
import os
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
os.environ["OPENAI_API_KEY"] = "YOUR OPEN API KEY"

template='''In an easy way translate the following sentence '{sentence}' into {target_language}'''
llm=OpenAI(temperature=0.7)
language_prompt = PromptTemplate(
    input_variables=["sentence",'target_language'],
    template=template,
)
language_prompt.format(sentence="How are you",target_language='hindi')
chain2=LLMChain(llm=llm,prompt=language_prompt)

chain2({'sentence':"I am vinay",'target_language':'hindi'})

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000167214180D0>: Failed to resolve 'api.openai.com' ([Errno 11001] getaddrinfo failed)")).
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000016721409350>: Failed to resolve 'api.openai.com' ([Errno 11001] getaddrinfo failed)")).
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLim

RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [ ]:
kgjhvhgj